In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import copy
import time
from tqdm import tqdm
import os

In [2]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split

import timm
from timm.loss import LabelSmoothingCrossEntropy
from timm.data import create_transform

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import models

In [3]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [4]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.1),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
#             transforms.RandomErasing(p=0.25, value='random')
        ])
        all_data = datasets.ImageFolder(data_dir, transform=transform)
        train_data_len = int(len(all_data)*0.75)
        valid_data_len = int((len(all_data) - train_data_len)/2)
        test_data_len = int(len(all_data) - train_data_len - valid_data_len)
        train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, train_data_len
    else:
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        all_data = datasets.ImageFolder(data_dir, transform=transform)
        train_data_len = int(len(all_data)*0.70)
        valid_data_len = int((len(all_data) - train_data_len)/2)
        test_data_len = int(len(all_data) - train_data_len - valid_data_len)
        train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return (val_loader, test_loader, valid_data_len, test_data_len)

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
dataset_path = "../../CUB_200_2011/images/"
(train_loader, train_data_len) = get_data_loaders(dataset_path, 256, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 64, train=False)
classes = get_classes(dataset_path)

In [7]:
dataloaders = {
    "train":train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train":train_data_len,
    "val": valid_data_len
}

In [8]:
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

print(train_data_len, test_data_len, valid_data_len)

35
28
28
8841 1769 1768


In [9]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, int(20/2), idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(classes[labels[idx]])

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [11]:
torch.backends.cudnn.benchmark = True
model = torchvision.models.efficientnet_b0(pretrained=True)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to C:\Users\杨大宇/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:01<00:00, 18.6MB/s]


In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [13]:
for param in model.parameters():
    param.requires_grad = False

n_inputs = model.classifier[1].in_features

model.classifier = nn.Sequential(
    nn.Linear(n_inputs,2048),
    nn.SiLU(),
    nn.Dropout(0.3),
    nn.Linear(2048, len(classes))
)

model = model.to(device)
print(model.classifier)

Sequential(
  (0): Linear(in_features=1280, out_features=2048, bias=True)
  (1): SiLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=2048, out_features=200, bias=True)
)


In [14]:
print(f"Trainable Params: {count_parameters(model)}")

Trainable Params: 3033288


In [15]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.2)
criterion = criterion.to(device)
optimizer = optim.AdamW(model.classifier.parameters(), lr=0.001)

In [16]:
step_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.96)

In [17]:
training_history = {'accuracy':[],'loss':[]}
validation_history = {'accuracy':[],'loss':[]}

In [18]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                training_history['accuracy'].append(epoch_acc)
                training_history['loss'].append(epoch_loss)
            elif phase == 'val':
                validation_history['accuracy'].append(epoch_acc)
                validation_history['loss'].append(epoch_loss)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [19]:
model_ft = train_model(model, criterion, optimizer, step_scheduler,
                       num_epochs=11)

Epoch 0/10
----------


100%|██████████| 35/35 [00:27<00:00,  1.27it/s]


train Loss: 4.5631 Acc: 0.1860


100%|██████████| 28/28 [00:14<00:00,  1.93it/s]


val Loss: 3.4009 Acc: 0.5011

Epoch 1/10
----------


...

val Loss: 2.2374 Acc: 0.8660

Epoch 10/10
----------


100%|██████████| 35/35 [00:23<00:00,  1.47it/s]


train Loss: 2.2087 Acc: 0.9118


100%|██████████| 28/28 [00:13<00:00,  2.03it/s]

val Loss: 2.2051 Acc: 0.8790

Training complete in 6m 53s
Best val Acc: 0.878959


In [20]:
torch.save(model.state_dict(), "cnn.pth")

In [21]:
torch.cuda.empty_cache()

In [22]:
test_loss = 0.0
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))

model_ft.eval()

for data, target in tqdm(test_loader):
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    with torch.no_grad():
        output = model_ft(data)
        loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == 64:
        for i in range(64):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(classes)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

100%|██████████| 28/28 [00:14<00:00,  1.89it/s]

Test Loss: 89.143082

Test Accuracy of 001.Black_footed_Albatross: 100% ( 8/ 8)
Test Accuracy of 002.Laysan_Albatross: 92% (13/14)
Test Accuracy of 003.Sooty_Albatross: 83% ( 5/ 6)
Test Accuracy of 004.Groove_billed_Ani: 100% ( 8/ 8)
Test Accuracy of 005.Crested_Auklet: 100% ( 4/ 4)
Test Accuracy of 006.Least_Auklet: 100% ( 7/ 7)
Test Accuracy of 007.Parakeet_Auklet: 100% ( 8/ 8)
Test Accuracy of 008.Rhinoceros_Auklet: 75% ( 6/ 8)
Test Accuracy of 009.Brewer_Blackbird: 100% ( 8/ 8)
Test Accuracy of 010.Red_winged_Blackbird: 100% ( 9/ 9)
Test Accuracy of 011.Rusty_Blackbird: 100% ( 2/ 2)
Test Accuracy of 012.Yellow_headed_Blackbird: 90% ( 9/10)
Test Accuracy of 013.Bobolink: 100% (12/12)
Test Accuracy of 014.Indigo_Bunting: 100% ( 8/ 8)
Test Accuracy of 015.Lazuli_Bunting: 92% (12/13)
Test Accuracy of 016.Painted_Bunting: 87% ( 7/ 8)
Test Accuracy of 017.Cardinal: 50% ( 2/ 4)
Test Accuracy of 018.Spotted_Catbird: 100% ( 5/ 5)
Test Accuracy of 019.Gray_Catbird: 100% ( 8/ 8)
Test Accuracy

In [23]:
from PIL import Image
from io import BytesIO
import requests


def apply_test_transforms(inp):
    out = transforms.functional.resize(inp, [224,224])
    out = transforms.functional.to_tensor(out)
    out = transforms.functional.normalize(out, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return out

In [24]:
def predict(model, filepath, show_img=False, url=False):
    if url:
        response = requests.get(filepath)
        im = Image.open(BytesIO(response.content))
    else:
         im = Image.open(filepath)
    if show_img:
        plt.imshow(im)
    im_as_tensor = apply_test_transforms(im)
    minibatch = torch.stack([im_as_tensor])
    if torch.cuda.is_available():
        minibatch = minibatch.cuda()
    pred = model(minibatch)
    _, classnum = torch.max(pred, 1)
    print(classnum)
    return classes[classnum]

In [25]:
def formatText(string):
    string = string[4:]
    string = string.replace("-", " ")
    return string

# 将这个链接换成对应的鸟的图片链接即可
formatText(predict(model_ft, "https://www.audubon.org/sites/default/files/styles/hero_cover_bird_page/public/web_h_andy-morffew_flickr-creative-common-by-2.0_altamiraoriole_flickr-3-adult.jpg?itok=ad9rnLPN", show_img=True, url=True))

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000018D1978D850>